In [22]:
import sys
import os

# 1. On récupère le chemin absolu du dossier parent (la racine du projet)
project_root = os.path.abspath('..')

# 2. On l'ajoute aux chemins que Python connaît (s'il n'y est pas déjà)
if project_root not in sys.path:
    sys.path.append(project_root)


In [52]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import torch
from sentence_transformers import util
from src.data_loader import load_data

In [3]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 2190.62it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [4]:
phrases = [
    "J'adore manger des muffins au chocolat",  # Phrase A
    "La recette du gâteau aux pommes est facile", # Phrase B
    "Il faut changer les pneus de la voiture"    # Phrase C
]

# On demande au modèle de transformer ces textes en vecteurs
embeddings = model.encode(phrases)

print(f"Nous avons transformé {len(phrases)} phrases.")
print(f"Chaque phrase est devenue un vecteur de taille : {embeddings.shape[1]}")

Nous avons transformé 3 phrases.
Chaque phrase est devenue un vecteur de taille : 384


In [9]:
embeddings

array([[-0.25525942, -0.51438767,  0.23558654, ...,  0.07950846,
         0.15474366, -0.08691271],
       [-0.04100963, -0.1802813 ,  0.18311352, ...,  0.3006359 ,
         0.54684293,  0.1799793 ],
       [-0.01210595,  0.2826209 ,  0.27153766, ...,  0.22718874,
         0.14208138,  0.2352363 ]], shape=(3, 384), dtype=float32)

In [10]:
# Calculons la similarité entre la phrase 1 (Muffin) et la 2 (Gâteau)
sim_muffin_gateau = util.cos_sim(embeddings[0], embeddings[1])

# Calculons la similarité entre la phrase 1 (Muffin) et la 3 (Voiture)
sim_muffin_voiture = util.cos_sim(embeddings[0], embeddings[2])


In [25]:
data = load_data(f"{project_root}/data/raw/mock_recipes.json")

Chargement des données depuis /Users/quentinhuet/Documents/Mines/COURS/3A-S1/nlp_course/rag-a-muffin/data/raw/mock_recipes.json...
Succès : 5 recettes chargées.


In [30]:
textes = [r['texte'] for r in data]

In [31]:
textes

["Titre : Muffins aux pépites de chocolat\nIngrédients : 200g de farine, 100g de sucre, 100g de beurre, 2 oeufs, 1/2 sachet de levure, 100g de pépites de chocolat, 10cl de lait.\nPréparation : 1. Mélanger les œufs et le sucre. 2. Ajouter la farine et la levure. 3. Fondre le beurre et l'ajouter avec le lait. 4. Incorporer les pépites. 5. Cuire 20 min à 180°C.",
 "Titre : Muffins Cœur Myrtille\nIngrédients : 250g de farine, 125g de sucre, 1 oeuf, 20cl de lait, 80ml d'huile végétale, 150g de myrtilles fraîches ou surgelées.\nPréparation : 1. Préchauffer le four à 200°C. 2. Dans un bol, mélanger farine, sucre et levure. 3. Dans un autre, battre l'oeuf, le lait et l'huile. 4. Mélanger les deux préparations rapidement (ne pas trop travailler la pâte). 5. Ajouter les myrtilles. 6. Remplir les moules et cuire 20-25 minutes.",
 'Titre : Quiche Lorraine Traditionnelle\nIngrédients : 1 pâte brisée, 200g de lardons fumés, 3 oeufs, 20cl de crème fraîche, lait, noix de muscade, sel, poivre.\nPrépara

In [32]:
embeddings = model.encode(textes)

In [37]:
n = len(textes)
similarities = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        similarities[i, j] = util.cos_sim(embeddings[i], embeddings[j])
similarities


array([[0.99999982, 0.86522245, 0.48893878, 0.51620317, 0.64628291],
       [0.86522245, 1.00000012, 0.61453044, 0.58521008, 0.72536552],
       [0.48893878, 0.61453044, 0.99999994, 0.55493045, 0.71662891],
       [0.51620317, 0.58521008, 0.55493045, 0.99999976, 0.5625267 ],
       [0.64628291, 0.72536552, 0.71662891, 0.5625267 , 1.        ]])

In [ ]:
def trouver_recette(question, model, textes, embeddings):
    question_embedding = model.encode(question)
    scores = util.cos_sim(question_embedding, embeddings)
    top_results = torch.topk(scores, k=3)
    return top_results

In [120]:
question1 = "Comment préparer un plat salé ?"
question2 = "Je veux un quelque chose sucré aux fruits."
question3 = "Quelle est la recette pour un petit gateau avec des pépites ?"

In [91]:
question_embedding = model.encode(question1)
scores = util.cos_sim(question_embedding, embeddings)
top_results = torch.topk(scores, k=3)
list(zip(list(top_results.values[0]), list(top_results.indices[0])))

[(tensor(0.6426), tensor(2)),
 (tensor(0.6103), tensor(4)),
 (tensor(0.6010), tensor(3))]

In [124]:
top = trouver_recette(question2, model, textes, embeddings)


In [126]:
top.values

tensor([[0.3384, 0.2897, 0.2876]])

In [128]:
list(zip(top.indices[0], top.values[0]))

[(tensor(2), tensor(0.3384)),
 (tensor(0), tensor(0.2897)),
 (tensor(3), tensor(0.2876))]

In [38]:
question = "Comment faire un repas salé ?"
question_embedding = model.encode(question)

In [54]:
scores = util.cos_sim(question_embedding, embeddings)
top_results = torch.topk(scores, k=1)

In [55]:
top_results

torch.return_types.topk(
values=tensor([[0.5688]]),
indices=tensor([[2]]))

In [62]:
score = top_results.values[0][0].item()
index = top_results.indices[0][0].item()

recette_trouvee = textes[index]

In [63]:
recette_trouvee

'Titre : Quiche Lorraine Traditionnelle\nIngrédients : 1 pâte brisée, 200g de lardons fumés, 3 oeufs, 20cl de crème fraîche, lait, noix de muscade, sel, poivre.\nPréparation : 1. Étaler la pâte. 2. Faire revenir les lardons. 3. Battre oeufs et crème, ajouter les lardons. 4. Verser sur la pâte. 5. Cuire 30 min à 200°C.'

In [57]:
top_results.values

tensor([[0.5688]])

[tensor([[0.5688]]), tensor([[2]])]

In [61]:
test2 = list(zip(top_results.values, top_results.indices))
test2

[(tensor([0.5688]), tensor([2]))]